# Ollaman käyttö

-Valtteri Airaksinen

---

### Virtuaaliympäristön luonti ja käyttö

Virtuaaliympäristön luonti:

`python -m venv llama`

Ympäristön käyttöönotto:

`source llama/Scripts/activate`

Pakettien lataaminen:

`pip install -r requirements.txt`

Ympäristön käytöstä poisto:

`deactivate`

---

### Ollaman käyttö

Käynnistys:

`ollama serve`

Haetaan mallit:

Avaa uusi terminaali -> Ota ympäristö käyttöön

Nyt voit valita minkä mallin haluat ja noutaa sen esimerkiksi komennolla `ollama pull llama3.2:3b`

Malleja:

`llama3.2:1b` - 1.3GB

`llama3.2:3b` - 2GB

`llama3.1:8b` - 4.7GB

`llama3.1:70b` - 40GB

`llama3.1:405b` - 229GB

`llama3:8b` - 4.7GB

`llama3:70b` - 40GB

Jos mallia ei spesifioi, noudetaan tuorein malli esim `llama3.2` -> `llama3.2:3b`

Näytä ladatut mallit:

`ollama list`

---

In [ ]:
from langchain_community.llms import Ollama

# Specify which model you want to use
llm = Ollama(model="llama3:latest")

# Chatbot
print(llm.invoke("Ymmärrätkö jos annan sinulle käsiteltäväksi suomenkielistä dataa jonka haluan sinun jaoittelevan?", stop=['<|eot_id|>']))


### Suomenkielen ymmärrys

Kysymys: Ymmärrätkö jos annan sinulle käsiteltäväksi suomenkielistä dataa jonka haluan sinun jaoittelevan?

---

`llama3.2:1b` Ehko, en ole yhtynyt kyseessä olevaan suomen kielen ja taidetta koskevaan tuotannoon. Tietoa on ainoastaan sijoiteltu lähemmäs sinua kuin muille joutuneille.

Tietoja ei ole vielä saatavilla, mutta oletettavasti voisin tarkistaa tietoa sitten kunhan käynnistyisi tehtäviä tai pyrkimyksiä. (14.5s)

---

`llama3.2:3b` Olen valmis käsittelemään suomenkielistä dataa! Olen koulutettu käsitelleämään erilaisia kieli- ja tekniikkatekijoita, mutta varsinkin suomalaisen kielen ja sen käyttöjärjestelmien ymmärrystä on olemassa vaikeuksia.

Käytän aina kielitietokantoja tai -tietoa esimerkiksi Wikipedia-sta, Valtion tutustumislainausverkkon tiedostoista, tai muuta luottavaa lähteerää. Kun sinulla on suomenkielinen data, olen valmis käsittelemään sitä.

Onko dataa kosketus- tai teksti-tyyppinen? Mikä on tarkoitusta? (28.6s)

---

`llama3:8b` Hei!

Kyseessä on suomalainen chatbot, joka on valmiina vastaanottamaan ja käsittämään suomenkielisiä tilastotietoja. Olen rakennettu oppimaan jaymmärtämään erilaisia kieliä, myös suomea.

Kuten haluat, anna minulle käsiteltäväksi suomenkielistä dataa, ja minä yritän tehdä siitä järkevää. Olkoon tietoa miten hyvin minä kykenen käsittämään sen! (46.2s)

In [ ]:
from langchain_community.llms import Ollama


# Function for extracting the values
def extractValuesFromText(llm, df):
    for index, row in df.iterrows():
        text = row['INFOTEKSTI']
        # Define the prompt for the model to extract specific information
        prompt = f"""
        Extract the following information from the text below, the text is in finnish:
        - Maalaji
        - Multavuus
        - pH
        - Nurmen ikä
        - Mitä viljellään
        - Lannoite
        - Lannoitemäärä

        Text: "{text}"

        Provide the output in this format:
        Maalaji: [extracted maalaji]
        Multavuus: [extracted multavuus]
        ph: [extracted pH]
        Nurmen ikä: [extracted nurmen ikä]
        Mitä viljellään: [extracted mitä viljellään]
        Lannoite: [extracted lannoite]
        Lannoitemäärä: [extracted lannoitemäärä]

        some helpful information:

        Here are all maalaji's, maalaji will always be one of these:
            "AS   aitosavi",
            "BCt  ruskosammalsaraturve",
            "CSt  sararahkaturve",
            "Ct   saraturve",
            "He   hiue",
            "HeS  hiuesavi",
            "HHk  hieno hiekka",
            "HHt  hieno hieta",
            "HkMr hiekka moreeni",
            "Hs   hiesu",
            "HsMr hiesumoreeni",
            "HsS  hiesusavi",
            "Ht   hieta",
            "HtMr hietamoreeni",
            "HtS  hietasavi",
            "Jm   järvimuta",
            "KHk  karkea hiekka",
            "KHt  karkea hieta",
            "LCt  metsäsaraturve",
            "Lj   lieju",
            "LjS  liejusavi, urpasavi",
            "LSt  metsärahkaturve",
            "Mm   multamaa",
            "Mt   muta",
            "SCt  rahkasaraturve",
            "SMr  savimoreeni",
            "Sr   sora",
            "SrMr soramoreeni",
            "St   rahkaturve"

        These are the different multavuus values, multavuus will always be one of these:
            vm    vähämultainen  
            m     multava  
            rm    runsasmultainen  
            erm   erittäin runsasmultainen  
            Mm    multamaa  
            
        Multavuus can be close to maalaji written together with it, for example rmHtMr. In this case:
            Maalaji: HtMr Hietamoreeni
            Multavuus: rm Runsasmultainen
            
        Answer to "Mitä viljellään?" is often a list of items with percentages on how much of them is in the field, for example:
            TT Tuure 35,5%, TT Rhonia 24,5%, NN Klaara 15%, RN Karolina 5%, Engl.rh Riikka10%, PA Saija 7,5%, VA Silvester 2,5%
            Here TT is Timotei, NN is Nurminata, RN is Ruokonata, Engl.rh is Englannin raiheinä, PA is Puna-apila, VA is Valko-apila and AA is Alsike-apila.
            Tuure, Rhonia, Klaara, Karolina, Riikka, Saija & Silvester are the brand names. Brand names might also not be specified for example:
            TT 20%, RN 80%
        There can also be just a single value like "Valio Carbo nurmiseos"

        Nurmen ikä is often a number and v or vuosi close to word nurmi, for example 2 v. nurmi. In this case:
            Nurmen ikä: 2v

        Lannoitus and lannoitusmäärä:
            for example from input: Lannoitus 14.5. YaraMila Y3 400 kg/ha.
            14.5 is the date (not to be extracted)
            Lannoitus: YaraMila Y3
            Lannoitusmäärä: 400 kg/ha

            Another example:  20.4. Se-salpietari 350 kg/ha.
            20.4. is the date (not to be extracted)
            Lannoitus: Se-salpietari
            Lannoitusmäärä: 350 kg/ha

        Some of the values we are looking for can be missing or have typos in them.
        If values are not found do not fill them in, just put None.
        No extra information is required from you, just the output in the requested format.
        """

        # Invoke the model with the prompt
        response = llm.invoke(prompt)

        # Print the result
        print(f"rivi  {int(index) + 1}")
        print("Input text: ", text)
        print("")
        print("Extracted Data: ", response)
        print("")


### Paranneltuja versioita funktiosta

In [1]:
from langchain_community.llms import Ollama

# Define the reusable portion of the prompt
prompt_template = """
Extract the following information from the text below, the text is in finnish:
- Maalaji
- Multavuus
- pH
- Nurmen ikä
- Mitä viljellään
- Lannoite
- Lannoitemäärä

Some helpful information:

Here are all maalaji's, maalaji will always be one of these:
    "AS   aitosavi",
    "BCt  ruskosammalsaraturve",
    "CSt  sararahkaturve",
    "Ct   saraturve",
    "He   hiue",
    "HeS  hiuesavi",
    "HHk  hieno hiekka",
    "HHt  hieno hieta",
    "HkMr hiekka moreeni",
    "Hs   hiesu",
    "HsMr hiesumoreeni",
    "HsS  hiesusavi",
    "Ht   hieta",
    "HtMr hietamoreeni",
    "HtS  hietasavi",
    "Jm   järvimuta",
    "KHk  karkea hiekka",
    "KHt  karkea hieta",
    "LCt  metsäsaraturve",
    "Lj   lieju",
    "LjS  liejusavi, urpasavi",
    "LSt  metsärahkaturve",
    "Mm   multamaa",
    "Mt   muta",
    "SCt  rahkasaraturve",
    "SMr  savimoreeni",
    "Sr   sora",
    "SrMr soramoreeni",
    "St   rahkaturve"

These are the different multavuus values, multavuus will always be one of these:
    vm    vähämultainen  
    m     multava  
    rm    runsasmultainen  
    erm   erittäin runsasmultainen  
    Mm    multamaa  

Multavuus can be close to maalaji written together with it, for example rmHtMr. In this case:
    Maalaji: HtMr Hietamoreeni
    Multavuus: rm Runsasmultainen
    
Answer to "Mitä viljellään?" is often a list of items with percentages on how much of them is in the field, for example:
    TT Tuure 35,5%, TT Rhonia 24,5%, NN Klaara 15%, RN Karolina 5%, Engl.rh Riikka10%, PA Saija 7,5%, VA Silvester 2,5%
    Here TT is Timotei, NN is Nurminata, RN is Ruokonata, Engl.rh is Englannin raiheinä, PA is Puna-apila, VA is Valko-apila and AA is Alsike-apila.
    Tuure, Rhonia, Klaara, Karolina, Riikka, Saija & Silvester are the brand names. Brand names might also not be specified for example:
    TT 20%, RN 80%
There can also be just a single value like "Valio Carbo nurmiseos"

Nurmen ikä is often a number and v or vuosi close to word nurmi, for example 2 v. nurmi. In this case:
    Nurmen ikä: 2v

Lannoitus and lannoitusmäärä:
    for example from input: Lannoitus 14.5. YaraMila Y3 400 kg/ha.
    14.5 is the date (not to be extracted)
    Lannoitus: YaraMila Y3
    Lannoitusmäärä: 400 kg/ha

    Another example:  20.4. Se-salpietari 350 kg/ha.
    20.4. is the date (not to be extracted)
    Lannoitus: Se-salpietari
    Lannoitusmäärä: 350 kg/ha

Some of the values we are looking for can be missing or have typos in them.
If values are not found do not fill them in, just put None.

Provide the output in this format:
Maalaji: [extracted maalaji]
Multavuus: [extracted multavuus]
ph: [extracted pH]
Nurmen ikä: [extracted nurmen ikä]
Mitä viljellään: [extracted mitä viljellään]
Lannoite: [extracted lannoite]
Lannoitemäärä: [extracted lannoitemäärä]

No extra information is required from you, just the output in the requested format.
"""

In [ ]:
# Function for extracting the values
# ~50% faster than the original
def extractValuesFromText(llm, df):
    for index, row in df.iterrows():
        text = row['INFOTEKSTI']
        
        # Combine the reusable template with the input text
        full_prompt = f"{prompt_template}\n\nText: \"{text}\""

        # Invoke the model with the combined prompt
        response = llm.invoke(full_prompt)

        # Print the result
        print(f"Row {int(index) + 1}")
        print("Input text: ", text)
        print("")
        print("Extracted Data: ", response)
        print("")


In [8]:
# Function for batch processing
def extractValuesFromTextBatch(llm, df, batch_size=5):
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        texts = batch['INFOTEKSTI'].tolist()

        # Create a single prompt for all rows in the batch
        batch_prompt = prompt_template + "\n\n"
        for idx, text in enumerate(texts):
            batch_prompt += f"Text {idx + 1}: \"{text}\"\n"

        # Add a clear instruction for batch output formatting
        batch_prompt += "\nProvide output in the format:\n"
        batch_prompt += "\n".join(
            [f"Text {j + 1}: Input: \"{{Original text}}\" Extracted Data: {{Extracted data}}" for j in range(len(texts))]
        )

        # Invoke the model with the batched prompt
        response = llm.invoke(batch_prompt)

        # Print the result
        print(f"Batch {i // batch_size + 1} Results:")
        print(f"Input: {texts[i]}")
        print(response)
        print("")


In [13]:
import pandas as pd

# Creating very tiny test sample
df = pd.read_excel('data/Korjuuaikanäytteiden_lisätiedot_16082024.xlsx')
#test_sample = df.head(5)
#test_sample = df.tail(5)
test_sample = df.sample(n=5)
test_sample.reset_index(inplace=True)

In [10]:
# Define the model you are using
llm = Ollama(model="llama3:latest")  # You can choose the model you want

# Run the function
#extractValuesFromText(llm, test_sample)
extractValuesFromTextBatch(llm, test_sample, 5)

Batch 1 Results:
Input: 3 v. nurmi. 70 kg  N/ha. Ei lietettä eikä täydennyskylvetty.  Valio Carbo nurmiseos.
Here are the extracted outputs in the requested format:

**Text 1**
Input: "3 v. nurmi. 70 kg N/ha. Ei lietettä eikä täydennyskylvetty. Valio Carbo nurmiseos."
Extracted Data:
Maalaji: None
Multavuus: None
pH: None
Nurmen ikä: 3 v.
Mitä viljellään: Valio Carbo nurmiseos
Lannoite: None
Lannoitemäärä: None

**Text 2**
Input: "Koulutila. TT 80%, NN 20%. Lannoitus 14.5. YaraMila Y3 400 kg/ha. Maalaji HHt pH 6,2. 3v. nurmi"
Extracted Data:
Maalaji: HHt
Multavuus: None
pH: 6,2
Nurmen ikä: 3 v.
Mitä viljellään: TT 80%, NN 20%
Lannoite: YaraMila Y3
Lannoitemäärä: 400 kg/ha

**Text 3**
Input: "2 v. nurmi. HiueS. Ei lietettä keväällä. Salpietari määrä? Carbo 3 niiton seos TT Rubinia 35%+ Uula 15%, RuoN Karolina 20%, NN Valtteri/Evertti 15% + apiloita"
Extracted Data:
Maalaji: HeS
Multavuus: None
pH: None
Nurmen ikä: 2 v.
Mitä viljellään: Carbo 3 niiton seos TT Rubinia 35%+ Uula 15%, RuoN 

In [12]:
extractValuesFromTextBatch(llm, test_sample, 5)

Batch 1 Results:
Input: 2 v nurmi. HeS. Lajikkeet: Grindstad 10 kg, Rhonia 12 kg, Klaara 8 kg, Riikka 3 kg, Karoliina 3 kg/ha. Lietettä 5.5. 10 t/ha, 20.4. Se-salpietari 350 kg/ha.
Here are the extracted outputs:

**Text 1**
Input: "2 v nurmi. HeS. Lajikkeet: Grindstad 10 kg, Rhonia 12 kg, Klaara 8 kg, Riikka 3 kg, Karoliina 3 kg/ha. Lietettä 5.5. 10 t/ha, 20.4. Se-salpietari 350 kg/ha."
Extracted Data:
Maalaji: HeS
Multavuus: m (multava)
ph: None
Nurmen ikä: 2 v
Mitä viljellään: Grindstad 10 kg, Rhonia 12 kg, Klaara 8 kg, Riikka 3 kg, Karoliina 3 kg/ha
Lannoite: Se-salpietari
Lannoitemäärä: 350 kg/ha

**Text 2**
Input: "Grindstad-timotei, Minto-nurminata"
Extracted Data:
Maalaji: AS (aitosavi)
Multavuus: None
ph: None
Nurmen ikä: None
Mitä viljellään: Grindstad-timotei, Minto-nurminata
Lannoite: None
Lannoitemäärä: None

**Text 3**
Input: "Timotei Tryggve, nurminata SW Minto"
Extracted Data:
Maalaji: AS (aitosavi)
Multavuus: None
ph: None
Nurmen ikä: None
Mitä viljellään: Timotei Tryg

In [14]:
extractValuesFromTextBatch(llm, test_sample, 5)

Batch 1 Results:
Input: Koulutila. Etelän paalinurmiseos. Lannoitus 8.5. YaraMila (25-3-8) 380 kg/ha. Maalaji m HHt. 3v. nurmi
Here is the extracted data in the requested format:

Text 1:
Input: "Koulutila. Etelän paalinurmiseos. Lannoitus 8.5. YaraMila (25-3-8) 380 kg/ha. Maalaji m HHt. 3v. nurmi"
Extracted Data:
Maalaji: HHt
Multavuus: m
pH: None
Nurmen ikä: 3v
Mitä viljellään: None
Lannoite: YaraMila (25-3-8)
Lannoitemäärä: 380 kg/ha

Text 2:
Input: "TT Nuutti 66%, NN Kasper 15%, PA 10%. Lannoitus Yara Y4 hiven 480 kg/ha. Maalaji rm HHt"
Extracted Data:
Maalaji: HHt
Multavuus: rm
pH: None
Nurmen ikä: None
Mitä viljellään: TT Nuutti 66%, NN Kasper 15%, PA 10%
Lannoite: Yara Y4 hiven
Lannoitemäärä: 480 kg/ha

Text 3:
Input: "Luomu-seos;TT Nuutti 25%,TT Tryggve 25%,SW Ares puna-apila 15%,Riikka  eng.raiheinä%,sinimailanen Creno 19%,koiranheinä Treposno 5%,sinimailanen Plato 1%"
Extracted Data:
Maalaji: None
Multavuus: None
pH: None
Nurmen ikä: None
Mitä viljellään: TT Nuutti 25%,TT Try